#What can governments and international organizations do right away to support workers and firms to navigate the COVID-19 crisis, #preserve jobs and incomes and prepare the way for a rapid recovery?

As the COVID-19 (Coronavirus) pandemic spreads relentlessly across the globe, the spillover could be enormous, especially (particularly) in Low Income Countries (LICs) with rudimentary public health networks, fragile economies and incomplete social protection systems.

With the pandemic now starting to gather momentum in Africa, repercussions from the unprecedented downturn in the Organization for Economic Co-operation and Development (OECD) economies are already impacting jobs in the global south. Huge negative capital flows have rocked emerging economies. Attempted lockdowns have generated major disruptions in economies characterized by high levels of informality and by fractured, incomplete social protection systems. In such settings, measures intended to “flatten the curve” of viral transmission might have generated increased transmissions, as thousands of informal migrant workers have crowded together trying return home. And where such internal migrants play key roles in economic life, the disruption of labor supplies is affecting harvests and food production.

In the absence of adequate public policy responses, there is no guarantee that these effects will be limited to a transitory, “V shaped” downturn. The world economy could be facing a deep, “U” shaped recession, or even an “L – shaped” recession, with no obvious end point, like the great depression of the 1930s.

https://www.jobsanddevelopment.org/confronting-the-jobs-impacts-of-covid-19-coronavirus/

![](https://www.jobsanddevelopment.org/wp-content/uploads/2020/04/covid-19.png)https://www.jobsanddevelopment.org/confronting-the-jobs-impacts-of-covid-19-coronavirus/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from colorama import Fore, Style

nRowsRead = 1000 # specify 'None' if want to read whole file
# ham_lyrics.csv has 3634 rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('../input/hackathon/task_2-COVID-19-death_cases_per_country_after_fifth_death-till_22_September_2020.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'task_2-COVID-19-death_cases_per_country_after_fifth_death-till_22_September_2020.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
print(Fore.BLUE + 'Data shape: ',Style.RESET_ALL,df.shape)
df.head()

In [ ]:
df.isnull().sum()

#Handling Missing Values

In [ ]:
# categorical features with missing values
categorical_nan = [feature for feature in df.columns if df[feature].isna().sum()>0 and df[feature].dtypes=='O']
print(categorical_nan)

In [ ]:
# replacing missing values in categorical features
for feature in categorical_nan:
    df[feature] = df[feature].fillna('None')

In [ ]:
df[categorical_nan].isna().sum()

In [ ]:
# Lets handle numerical features with nan value
numerical_nan = [feature for feature in df.columns if df[feature].isna().sum()>1 and df[feature].dtypes!='O']
numerical_nan

In [ ]:
## Replacing the numerical Missing Values

for feature in numerical_nan:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    
    df[feature].fillna(median_value,inplace=True)
    
df[numerical_nan].isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBRFClassifier,XGBClassifier

#Codes by https://www.kaggle.com/nalkrolu/machine-learning-application-with-iris-dataset/notebook

In [ ]:
df_encode = df.copy()

X = df.drop(["stringency_index_10_days_after_fifth_death"],axis=1)
y = df_encode["stringency_index_10_days_after_fifth_death"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=0)

In [ ]:
from sklearn.model_selection import cross_val_score
def CrossValidationScore(model_list):
    global X,y
    
    mean_cross_val_score = []
    model_name           = []
    
    for model in model_list:
        model_name.append(type(model).__name__)
        
    for i in model_list:
        scores = cross_val_score(i, X, y, cv=5)
        mean_cross_val_score.append(scores.mean())
        
    cvs = pd.DataFrame({"Model Name":model_name,"CVS":mean_cross_val_score})
    return cvs.style.background_gradient("Greens")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from yellowbrick.classifier import ROCAUC,ConfusionMatrix

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())

#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
        
print('Labelling done.')

In [ ]:
df = pd.get_dummies(df)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50,100,50),activation="relu",max_iter=10000,solver="adam")
mlp.fit(X_train,y_train)

plt.figure(figsize=(10,5))
visualizer = ROCAUC(mlp, classes=[0,1,2]);
visualizer.fit(X_train, y_train)      
visualizer.score(X_test, y_test)        
visualizer.show();

plt.figure(figsize=(5,4))
cm = ConfusionMatrix(mlp, classes=[0,1,2])
cm.fit(X_train, y_train)
cm.score(X_test, y_test)
cm.show();

#Another ML model that I could NOT perform (including Confusion Matrix and ROC curves).

In [ ]:
px.bar(df, x = 'stringency_index_10_days_after_fifth_death', y = 'date_fifth_death', color = 'country_name',orientation='h' , title='Stringency Measures after fifth death by Covid19',  height = 500 )

#Codes by Xhlulu https://www.kaggle.com/xhlulu/android-or-ios-data-exploration/comments

In [ ]:
px.histogram(df, x='deaths_per_million_10_days_after_fifth_death', range_x=[-5, 50], color='date_fifth_death')

In [ ]:
px.histogram(df, x='stringency_index_10_days_after_fifth_death', range_x=[-5, 50], color='date_fifth_death')

In [ ]:
px.pie(df, 'date_fifth_death')

In [ ]:
px.pie(df, 'stringency_index_10_days_after_fifth_death')

In [ ]:
#Code by Xhlulu   https://www.kaggle.com/xhlulu/android-or-ios-data-exploration/comments

px.scatter(df, x='date_fifth_death', y='stringency_index_10_days_after_fifth_death', color='country_name', log_y=True, marginal_y='box', range_x=[-20, 200])

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#2B3A67','#42a7f5','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke was Here.' )